# Quickstart: Using IBM Granite on IBM watsonx.ai

This notebook is a step-by-step tutorial for running **Granite** models on **IBM watsonx.ai** from your Jupyter environment. It covers prerequisites, how to get your credentials and project set up in IBM Cloud, local installation, environment configuration, choosing the right model, and a minimal "Hello World" via both the **ibm-watsonx-ai** SDK and **LangChain**.

**What you'll do:**
1. Complete IBM Cloud prerequisites (account setup, API key, watsonx.ai project).
2. Install the required Python packages.
3. Choose a **Granite** model (e.g., `granite-3-3-8b-instruct`).
4. Make a generation request with the **IBM SDK**.
5. Make the same request via **LangChain**.

> References: [watsonx.ai foundation model catalog](https://www.ibm.com/products/watsonx-ai/foundation-models), [Granite 3.3 docs](https://www.ibm.com/granite/docs/models/granite/).


## 0) Prerequisites

Before running any code here, complete these steps in IBM Cloud:

### Create / sign in to an IBM Cloud account and provision watsonx.ai.  
   - Go to [IBM Cloud](https://cloud.ibm.com) and sign up or log in with your IBM ID.
   - If you’re new, you can start with a Lite plan or apply a promo code for free credits.
   - Create a watsonx.ai service instance from your IBM Cloud dashboard. This provides access to [IBM foundation models](https://www.ibm.com/products/watsonx-ai/foundation-models).
   - After login, navigate to the IBM watsonx.ai service from the IBM Cloud Catalog. You can also create a watsonx.ai free trial by following these [instructions](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/signup-wx.html?context=wx&audience=wdp#personal).
   - Go to Catalog and select the `AI & Machine Learning` option. Next, choose `watsonx.ai`.

   ![IBMCloud](./recordings/ibm-cloud-wx-launch.gif "segment")

### Create a watsonx.ai Project
   A watsonx.ai project is a collaborative workspace within IBM's [watsonx.ai](https://dataplatform.cloud.ibm.com/wx/home?context=wx) platform, where teams build, train, tune, and deploy both generative AI and traditional machine learning models. A project in watsonx.ai acts as a shared environment for data scientists, developers, and other collaborators.

   ![WXProjectCreation](./recordings/wx-create-new-project.gif "segment")

   - On the watsonx.ai console, click on `Projects` and select `View all projects` and choose a [New project](https://dataplatform.cloud.ibm.com/projects/new-project?context=wx).
   - Choose:
      - Empty project (start from scratch), or
      - Sample project (with preloaded assets), or
      - Import project (upload a .zip from another environment).
   - Name your project (must be unique, 1–255 chars, no %, \\, or leading/trailing spaces).
   - Storage requirement: Associate an IBM Cloud Object Storage instance (create one if you don’t have it).
   - Click Create. Your project now has:
      - A unique Project ID (needed for API calls).
      - A dedicated storage bucket for assets.
   - Note your service **URL** (region endpoint), e.g., `https://us-south.ml.cloud.ibm.com`.

   For working with existing projects, see here:
   ![WXProjectAccess](./recordings/wx-open-existing-project.gif "segment")

### Associate Watson Machine Learning Service
   Watson Machine Learning provides a full range of tools and services, so you can build, train, and deploy Machine Learning models.

   - Open your project, in WML and click on `Manage` at the top nav bar selecting `Services and Integrations`.
   - Click `Associate Service` and choose `Watson Machine Learning (WML)`. Note: You must have Admin role in project.
   - If you don’t have a WML instance:
      - Go to IBM Cloud Catalog, select Watson Machine Learning.
      - Create an instance in the same region as your project.
      - Once created, link the WML instance to your project.
      - This step is required for running foundation models and APIs.

   ![AssociateWML](./recordings/associate-wml.gif "segment")

### Obtain Your Credentials
   - In IBM Cloud, go to Manage option and select `Access (IAM)` and choose [API keys](https://cloud.ibm.com/iam/apikeys).
   - Create an API key (or use an existing one).
   - Download or copy your key immediately (cannot see again after creation).
   - Note:
      - API Key : this is used for authentication.
      - Service URL. E.g., https://us-south.ml.cloud.ibm.com.
      - Project ID. This can be obtained from your watsonx.ai project details. In watsonx.ai, open the left menu and select `Projects` > `View all projects`. Select your desired project.
      - Go to Manage tab, click on General. Don't forget to copy Project ID for later.
         - You should see Watson Machine Learning listed under Services for your project.
         - These three values (API_KEY, URL, PROJECT_ID) are required for SDK and LangChain integration.
         - See: [Credentials for programmatic access](https://www.ibm.com/docs/en/watsonx/saas?topic=resources-credentials-programmatic-access). 

   ![IAMKey](./recordings/iam-key-demo.gif "segment")

### Validate Access
   -  Ensure your IBM Cloud user has Editor role for watsonx.ai and WML services.
   - If you plan to use APIs directly, confirm you can generate a [Bearer token](https://cloud.ibm.com/docs/account?topic=account-iamtoken_from_apikey) using your API key.

### (Optional) In **Resource hub** or **Prompt Lab**: 
   The Watsonx Prompt Lab is a tool within IBM's watsonx.ai platform that allows users to experiment with, test, and refine prompts for various large language models (LLMs).

   - Review Granite 3.3 models and copy the exact **`model_id`**. You can also list available models via API (shown later).  
   - See: [Get foundation model information / model IDs](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-api-model-ids.html?context=wx&audience=wdp).

> You will paste **three values** into this notebook: `WATSONX_URL`, `WATSONX_APIKEY`, and `WATSONX_PROJECT_ID`.


## 1) Choosing a Granite model

The Granite models available on watsonx are listed on the [Resource Hub](https://dataplatform.cloud.ibm.com/samples?context=wx&tab=foundation-model&query=granite). For more information about Granite models on watsonx, see [Foundation models](https://www.ibm.com/products/watsonx-ai/foundation-models) on watsonx.ai.

| Model Type | Model IDs | Best For | Notes |
|------------|-----------|----------|-------|
| **Instruct** | `granite-3-3-2b-instruct`, `granite-3-3-8b-instruct` | Chat, Q&A, reasoning, summaries, coding | Use **8B** for higher quality; **2B** is faster and cheaper |
| **Base**    | `granite-3-3-2b-base`, `granite-3-3-8b-base`         | Fine-tuning, advanced prompting         | Pre-trained only; not instruction-tuned |

**Context Support**: All Granite 3.3 models support up to **128K tokens**.


> ⚠️ Availability can be **region-specific**; if a model isn’t found, check your region’s catalog or list models via API (Section 4).


## 2) Install packages locally

Run the next cell to install the SDKs and helpers:


In [ ]:
%pip install ibm-watsonx-ai langchain langchain-ibm git+https://github.com/ibm-granite-community/utils

## 3) Configure your local environment

Set the required settings as environment variables for this session. (You can also facilitate reuse across sessions, by maintaining an `.env` file in you local execution.)

You need to provide to WatsonxLLM three values which are specific to your configuration:

1. `WATSONX_PROJECT_ID` - the ID of the project you have associated to the WatsonMachineLearning service.
2. `WATSONX_URL` - the endpoint which is applicable to your watsonx instance. More details on developer access can be found [here](https://dataplatform.cloud.ibm.com/developer-access?context=wx).
3. `WATSONX_APIKEY` - the key which authenticates your request
To access all three, go to your [watsonx dashboard](https://dataplatform.cloud.ibm.com/) and scroll down to Developer Access.

Pick your project from the dropdown and the UI will provide both the Project ID and the watsonx.ai URL.

There are also options to either create a new API key or manage existing ones.

More in-depth instructions around API keys can be found in [this document](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).


## 4) Hello World with the **ibm-watsonx-ai** SDK

A minimal generation call with a Granite 3.3 Instruct model.  
Docs: [ibm-watsonx-ai Python library](https://www.ibm.com/docs/en/watsonx/saas?topic=resources-python-library)


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

params = {
    "temperature": 0.7,
    "top_p": 0.9,
    "max_tokens": 128,
}

creds = Credentials(
    url=get_env_var("WATSONX_URL"),
    api_key=get_env_var("WATSONX_APIKEY"),
)

chat = ModelInference(
    model_id="ibm/granite-3-3-8b-instruct",  # set to 2B if desired
    credentials=creds,
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=params,
)

# Chat-style messages
messages = [
    {"role": "user", "content": "Say hello to the world in one short sentence."}
]

result = chat.chat(messages=messages)

try:
    print("\n--- Assistant ---\n")
    print(result["choices"][0]["message"]["content"])
except Exception:
    pass


## 5) Hello World with **LangChain**

LangChain’s `ChatWatsonx` wrapper uses your watsonx **URL**, **project_id**, and **model_id**.  
Docs: [LangChain watsonx integration](https://python.langchain.com/docs/integrations/llms/ibm_watsonx/), [langchain-ibm (PyPI)](https://pypi.org/project/langchain-ibm/), [ChatWatsonx](https://python.langchain.com/api_reference/ibm/chat_models/langchain_ibm.chat_models.ChatWatsonx.html)


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.utils import convert_to_secret_str
from langchain_ibm.chat_models import ChatWatsonx

# Set decoding parameters
model_parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 64,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1.0,
}

# Initialize ChatWatsonx
llm = ChatWatsonx(
    model_id="ibm/granite-3-3-8b-instruct",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=model_parameters,
)

# Use ChatPromptTemplate with HumanMessage
prompt = ChatPromptTemplate.from_messages([
  HumanMessage("Hello, world! Pretend you're a robot greeting humans for the first time. What would you say?")
])

# LangChain pipeline
chain = prompt | llm | StrOutputParser()

# Invoke the chain
print(chain.invoke({}))